[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

In [ ]:
%cd tutorial

El archivo *src/14/urls.py* contiene lo siguiente:

``` python 
from django.urls import path, re_path
from . import views, endpoint_views

urlpatterns = [path('', views.vista),
               path('carga', views.carga),
               re_path(r'^(?P<clave>[0-9]{4}$)', endpoint_views.clave),]
```

In [ ]:
%cp ../src/14/urls.py api/urls.py

El archivo *src/14/endpoint_views.py* contiene lo siguiente:

``` python
from . import models
from django.views.decorators.csrf import csrf_exempt
from django.http import JsonResponse, HttpResponseNotFound, HttpResponseBadRequest

campos = {'numero_de_cuenta':int, 'nombre':str, 'primer_apellido':str,
          'segundo_apellido':str, 'carrera':str, 'semestre':int, 'promedio':float,
          'al_corriente':bool}

@csrf_exempt
def clave(request, clave):
    '''Función de vista que define un endpoint correspondiente a una clave de 4 dígitos.
    Opera con los métodos GET, POST, DELETE.'''
    
    # Cuando la petición es GET va a obtener los datos del alumno con la clave correrspondiente.
    if request.method == "GET":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave) 
            return JsonResponse({campo:getattr(alumno, campo) for campo in campos})
        # La excepción models.Alumno.DoesNotExist se desencadena cuando la búsqueda no arroje un resultado.
        except models.Alumno.DoesNotExist:
            return HttpResponseNotFound()
    # Cuando la petición es POST va a dar de alta los datos del alumno con la clave correspondiente y los datos enviados.
    if request.method == "POST":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave) 
            return HttpResponseBadRequest()
        except models.Alumno.DoesNotExist:
            registro = request.POST.dict()
            registro["numero_de_cuenta"] = int(clave)
            objeto = models.Alumno()
            for campo in registro:
                setattr(objeto, campo, registro[campo])
            objeto.save()
            return JsonResponse(registro)
    # Cuando la petición es DELETE el alumno es eliminado de la base de datos.
    if request.method == "DELETE":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave)
            alumno.delete()
            return JsonResponse({'estado': 'eliminado'})   
        except models.Alumno.DoesNotExist:
            return HttpResponseNotFound()
```


In [ ]:
%cp ../src/14/endpoint_views.py api/endpoint_views.py

In [ ]:
%cat api/endpoint_views.py

In [ ]:
!./manage.py runserver 0.0.0.0:5000

http://localhost:5000/api/1221

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>